inputs

In [49]:
## inputs
gene_of_interest = 51816 # ADA2 entrez id
input_protein_file = "ADA2.txt"
mutations_of_interest = ["G47A", "Y453C"]

## optional inputs
input_species_id = 9606 # human
output_species_id = 7227 # fly

#file paths
ortholog_script = "pull_diopt_orthologs.py"
ortholog_output_folder = "ortholog_output"
protein_info_script = "get_protein_info.py"

imports

In [50]:
## imports
import os
import pandas as pd

functions

In [51]:
## functions
def iterate_files_in_folder(folder_path):
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

code

In [ ]:
## get orthologs
os.system(f"python {ortholog_script} {input_species_id} {output_species_id} {gene_of_interest}")

In [66]:
## get orthologs gene ids from output folder called otholog_outputs
ortholog_list = iterate_files_in_folder(ortholog_output_folder)

## get best ortholog matches
## filtering criteria: best match, reverse match with high score, or high confidence
all_entrez_ids = set()  # Initialize a set to store unique entrez ids

for file in ortholog_list:
    ortholog_df = pd.read_csv(file)

    # results with high confidence
    high_confidence_entrez_ids = ortholog_df[ortholog_df['confidence'] == "high"]['entrez_id'].tolist()
    all_entrez_ids.update(high_confidence_entrez_ids)

    # results with highest score
    best_score_entrez_ids = ortholog_df[ortholog_df['best_score'] == "Yes"]['entrez_id'].tolist()
    all_entrez_ids.update(best_score_entrez_ids)

    # results with highest reverse score and within five points of the results with highest score
    best_score = ortholog_df[ortholog_df['best_score'] == "Yes"]['score']
    filtered_df = ortholog_df[ (ortholog_df['best_score_rev'] == "Yes") &
        (ortholog_df['score'].apply(lambda x: any(abs(x - bs) <= 5 for bs in best_score)))]
    best_reverse_entrez_ids = filtered_df['entrez_id'].tolist()
    all_entrez_ids.update(best_reverse_entrez_ids)

# Convert set to space-separated string
all_entrez_ids_str = " ".join(map(str, all_entrez_ids))

# Convert set to list if needed
# all_entrez_ids = list(all_entrez_ids)

In [68]:
file1 = "protein_orthologs.fasta"
file2 = "ADA2.fasta"
combined_file = "combined_proteins.fasta"

## get protein info
os.system(f"python {protein_info_script} {all_entrez_ids_str} protein_orthologs.zip {file1}")
# os.system(f"python {protein_info_script} {gene_of_interest} input_protein.zip {file2}")

## combine into one file
with open(file1, 'r') as f1:
    data1 = f1.read()
with open(file2, 'r') as f2:
    data2 = f2.read()
with open(combined_file, 'w') as cf:
    cf.write(data1)
    cf.write("\n")
    cf.write(data2)

## submit clustal-w script
os.system(f"sbatch clustalo.sh -INFILE={combined_file}")

0